# CS178 Project

In [95]:
# import library, module and training/evaluation data

import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from mltools import dtree
from mltools import cluster
import mltools.logistic2 as lc2
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.cross_validation import KFold
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.linear_model import RidgeCV

# Note: file is comma-delimited
X = np.genfromtxt("Data/kaggle.X1.train.txt",delimiter=',')
Y = np.genfromtxt("Data/kaggle.Y.train.txt",delimiter=',')
Xtest = np.genfromtxt("data/kaggle.X1.test.txt",delimiter=',')

In [96]:
n_trees = 30
clfs = [
        ExtraTreesRegressor(n_estimators = n_trees *2),
        RandomForestRegressor(n_estimators = n_trees),
        GradientBoostingRegressor(n_estimators = n_trees)
    ]

In [97]:

X1,X2,Y1,Y2 = ml.splitData(X,Y,0.75)
temp_results = np.zeros((  len(Y2)   ,len(clfs)    ))
test1=np.zeros((  Xtest.shape[0]   ,len(clfs)    ))
for i, clf in enumerate(clfs):
    clf.fit(X1,Y1)
    temp_results[:,i] = clf.predict(X2)
    test1[:,i] = clf.predict(Xtest)

In [98]:
alphas = [0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0, 500.0, 1000.0]

bclf = RidgeCV(alphas=alphas, normalize=True, cv=5)
bclf.fit(temp_results, Y2)


RidgeCV(alphas=[0.0001, 0.005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0, 500.0, 1000.0],
    cv=5, fit_intercept=True, gcv_mode=None, normalize=True, scoring=None,
    store_cv_values=False)

In [99]:

Ye = bclf.predict(test1)
fh = open('predictionsP3.csv','w')    # open file for upload
fh.write('ID,Prediction\n')         # output header line
for i,yi in enumerate(Ye):
  fh.write('{},{}\n'.format(i+1,yi)) # output each prediction
fh.close()  

print 'Writing finished!'

Writing finished!
